In [156]:
import pandas as pd
import re
import numpy as np
import nltk

from prepare import basic_body_clean, tokenize, remove_stopwords, lemmatize, basic_code_clean
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

# ACQUIRE, PREP AND SPLIT DATA

In [157]:
gitMDs = pd.read_json('gitMDsv2.json')

gitMDs['readme'] = gitMDs['body'].apply(basic_body_clean).apply(tokenize).apply(remove_stopwords).apply(lemmatize)
gitMDs['language'] = gitMDs['top_code'].apply(basic_code_clean).apply(tokenize).apply(remove_stopwords)
gitMDs[['language', 'percentage']] = gitMDs['language'].str.split(" ",expand=True)
gitMDs['percentage'] = pd.to_numeric(gitMDs['percentage'])

gitMDs.drop(columns = ['body', 'top_code', 'percentage'], inplace = True)

In [158]:
gitMDs.head()

,readme,language
0,freecodecamporg ' opensource codebase curricul...,javascript
1,supporting vuejs vuejs mitlicensed open source...,javascript
2,react react javascript library building user i...,javascript
3,bootstrap sleek intuitive powerful frontend fr...,javascript
4,airbnb javascript style guide mostly reasonabl...,javascript


In [159]:
train_validate, test = train_test_split(gitMDs[['language', 'readme']], 
                                        stratify=gitMDs.language, 
                                        test_size=.2)

train, validate = train_test_split(train_validate, 
                                   stratify=train_validate.language, 
                                   test_size=.25)

In [160]:
train.language.value_counts()

javascript    54
python        54
Name: language, dtype: int64

# SEPARATING LANGUAGES

In [ ]:
# function to convert df rows to list of words
def rows_to_lists(text):
    words = re.sub(r'[^\w\s]', '', text).split()
    return words

In [ ]:
# separating languages and converting rows to lists (1 list per language)
javascript = rows_to_lists(' '.join(train[train.language == 'javascript'].readme))
python = rows_to_lists(' '.join(train[train.language == 'python'].readme))

# BIGRAM FEATURE

In [12]:
# getting value counts of bigrams
# ie. how often did they appear in data
js_bigrams = pd.Series(nltk.ngrams(javascript, 2)).value_counts()

# creating top 10 series of most common spam bigrams
top_10_js_bigrams = js_bigrams.head(10)

# checking top 10
top_10_js_bigrams

(react, native)        787
(io, android)           73
(component, react)      56
(android, io)           49
(npm, install)          38
(best, practice)        34
(open, source)          34
(native, component)     33
(back, top)             33
(b, 2)                  33
dtype: int64

In [13]:
# getting value counts of bigrams
# ie. how often did they appear in data
py_bigrams = pd.Series(nltk.ngrams(python, 2)).value_counts()

# creating top 10 series of most common spam bigrams
top_10_py_bigrams = py_bigrams.head(5)

# checking top 10
top_10_py_bigrams

(1, 2)             49
(pip, install)     45
(2, 3)             45
(9, 9)             42
(3, 4)             32
(false, false)     27
(docker, run)      23
(true, set)        20
(python, 3)        20
(pull, request)    20
dtype: int64

In [16]:
pd.Series(nltk.ngrams(python, 2))

0                     (poetry, dependency)
1                 (dependency, management)
2                     (management, python)
3                         (python, poetry)
4                           (poetry, help)
                       ...                
33062                    (fzaninotto, php)
33063                         (php, faker)
33064                  (faker, distribute)
33065               (distribute, buildout)
33066    (buildout, modernpackagetemplate)
Length: 33067, dtype: object

In [18]:
top_10_py_bigrams.index.tolist()

[('1', '2'),
 ('pip', 'install'),
 ('2', '3'),
 ('9', '9'),
 ('3', '4'),
 ('false', 'false'),
 ('docker', 'run'),
 ('true', 'set'),
 ('python', '3'),
 ('pull', 'request')]

# SINGLE WORD FEATURE (JS)

In [161]:
# how often did each word appear in data?
js_words = pd.Series(nltk.ngrams(javascript, 1)).value_counts()

top_5_js_words = js_words.head(5)

t5jsl = top_5_js_words.index.tolist()

In [162]:
t5jsreduced = []

for x in t5jsl:
    t5jsreduced.append(x[0])
    
t5jsreduced

['react', 'native', 'javascript', '9', 'component']

In [163]:
contains = [train['readme'].str.contains(w) for w in t5jsreduced]

t5_js_index = train[np.any(contains, axis = 0)].index.tolist()

train['has_top_5_js_word'] = train.index.isin(t5_js_index)

train

,language,readme,has_top5_js_word
90,python,nginxproxy set container running nginx dockerg...,True
85,javascript,async utility module provides straightforward ...,True
65,javascript,build next vuejs application confidence using ...,False
31,javascript,html5 boilerplate html5 boilerplate profession...,True
68,javascript,api creation made simple secure fast advanced ...,True
...,...,...,...
42,javascript,svelte svelte new way build web application ' ...,True
35,javascript,momentjs javascript date library parsing valid...,True
22,javascript,socketio feature socketio enables realtime bid...,True
141,python,tensor2tensor tensor2tensor t2t short library ...,True


# SINGLE WORD FEATURE (PY)

In [165]:
# how often did each word appear in data?
py_words = pd.Series(nltk.ngrams(python, 1)).value_counts()

top_5_py_words = py_words.head(5)

t5pyl = top_5_py_words.index.tolist()

In [167]:
t5pyreduced = []

for x in t5pyl:
    t5pyreduced.append(x[0])
    
t5pyreduced

['python', 'use', 'install', 'file', 'not']

In [168]:
contains = [train['readme'].str.contains(w) for w in t5pyreduced]

t5_py_index = train[np.any(contains, axis = 0)].index.tolist()

train['has_top_5_py_word'] = train.index.isin(t5_py_index)

train

,language,readme,has_top5_js_word,has_top_5_py_word
90,python,nginxproxy set container running nginx dockerg...,True,True
85,javascript,async utility module provides straightforward ...,True,True
65,javascript,build next vuejs application confidence using ...,False,True
31,javascript,html5 boilerplate html5 boilerplate profession...,True,True
68,javascript,api creation made simple secure fast advanced ...,True,True
...,...,...,...,...
42,javascript,svelte svelte new way build web application ' ...,True,True
35,javascript,momentjs javascript date library parsing valid...,True,False
22,javascript,socketio feature socketio enables realtime bid...,True,True
141,python,tensor2tensor tensor2tensor t2t short library ...,True,True


# SINGLE WORD MODEL

In [173]:
y_train = train['language']

X_train = train[['has_top5_js_word', 'has_top_5_py_word']]

In [174]:
# from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(random_state=123)

logit.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [176]:
train['predicted'] = logit.predict(X_train)

In [177]:
print(classification_report(train.language, train.predicted))

              precision    recall  f1-score   support

  javascript       0.56      0.83      0.67        54
      python       0.67      0.33      0.44        54

    accuracy                           0.58       108
   macro avg       0.61      0.58      0.56       108
weighted avg       0.61      0.58      0.56       108



# BASELINE

In [25]:
train['baseline_prediction'] = 'python'

print(classification_report(train.language, train.baseline_prediction))

              precision    recall  f1-score   support

  javascript       0.00      0.00      0.00        54
      python       0.50      1.00      0.67        54

    accuracy                           0.50       108
   macro avg       0.25      0.50      0.33       108
weighted avg       0.25      0.50      0.33       108



# BAG OF WORDS
***

## TRAIN

In [ ]:
# creating vectorizer
vectorizer = CountVectorizer(stop_words='english', 
                             min_df=20, 
                             ngram_range=(1,2), 
                             binary=True)

# Learn vocabulary in sentences. 
vectorizer.fit(train.readme)

# Get dictionary. 
vectorizer.get_feature_names()

# Transform each sentences in vector space.
bow = vectorizer.transform(train.readme)

X_bow = bow

In [30]:
# create our y dataset
y = train['language']

lm = LogisticRegression().fit(X_bow, y)

train['predicted'] = lm.predict(X_bow)

In [31]:
print(classification_report(train.language, train.predicted))

              precision    recall  f1-score   support

  javascript       1.00      1.00      1.00        54
      python       1.00      1.00      1.00        54

    accuracy                           1.00       108
   macro avg       1.00      1.00      1.00       108
weighted avg       1.00      1.00      1.00       108



## VALIDATE

In [34]:
# Transform each sentences in vector space.
v_bow = vectorizer.transform(validate.readme)

validate['predicted'] = lm.predict(v_bow)

print(classification_report(validate.language, validate.predicted))

              precision    recall  f1-score   support

  javascript       0.88      0.83      0.86        18
      python       0.84      0.89      0.86        18

    accuracy                           0.86        36
   macro avg       0.86      0.86      0.86        36
weighted avg       0.86      0.86      0.86        36



## TEST

In [35]:
# Transform each sentences in vector space.
v_bow = vectorizer.transform(test.readme)

test['predicted'] = lm.predict(v_bow)

print(classification_report(test.language, test.predicted))

              precision    recall  f1-score   support

  javascript       0.71      0.67      0.69        18
      python       0.68      0.72      0.70        18

    accuracy                           0.69        36
   macro avg       0.70      0.69      0.69        36
weighted avg       0.70      0.69      0.69        36



# TFIDF
***

# TRAIN

In [38]:
tfidf = TfidfVectorizer(stop_words='english', min_df = 20,
                             ngram_range=(1,2), 
                             binary=True)

tfidf_sparse_matrix = tfidf.fit_transform(train.readme)

tfidf.vocabulary_

# Transform to document-term matrix
vector_spaces = tfidf.transform(train.readme)

X_tfidf = tfidf_sparse_matrix

In [39]:
lm_tfidf = LogisticRegression().fit(X_tfidf, y)

train['pred_tfidf'] = lm_tfidf.predict(X_tfidf)

print(classification_report(train.language, train.pred_tfidf))

              precision    recall  f1-score   support

  javascript       1.00      0.98      0.99        54
      python       0.98      1.00      0.99        54

    accuracy                           0.99       108
   macro avg       0.99      0.99      0.99       108
weighted avg       0.99      0.99      0.99       108



# VALIDATE

In [40]:
tfidf_sparse_matrix = tfidf.transform(validate.readme)

# Transform to document-term matrix
vector_spaces = tfidf.transform(validate.readme)

X_tfidf = tfidf_sparse_matrix

validate['pred_tfidf'] = lm_tfidf.predict(X_tfidf)

print(classification_report(validate.language, validate.pred_tfidf))

              precision    recall  f1-score   support

  javascript       0.94      0.83      0.88        18
      python       0.85      0.94      0.89        18

    accuracy                           0.89        36
   macro avg       0.89      0.89      0.89        36
weighted avg       0.89      0.89      0.89        36



# TEST

In [41]:
tfidf_sparse_matrix = tfidf.transform(test.readme)

# Transform to document-term matrix
vector_spaces = tfidf.transform(test.readme)

X_tfidf = tfidf_sparse_matrix

test['pred_tfidf'] = lm_tfidf.predict(X_tfidf)

print(classification_report(test.language, test.pred_tfidf))

              precision    recall  f1-score   support

  javascript       0.71      0.67      0.69        18
      python       0.68      0.72      0.70        18

    accuracy                           0.69        36
   macro avg       0.70      0.69      0.69        36
weighted avg       0.70      0.69      0.69        36

